# ECG Split 1D-CNN Client Side

This code is the server part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages

In [1]:
mode = 'CIFAR10'

import os
import struct
import socket
import pickle
import time

import h5py
import numpy as np

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
from torchvision.datasets import CIFAR10

## Define ECG dataset class

In [2]:
if mode == 'ECG':
    class ECG(Dataset):
        def __init__(self, train=True):
            if train:
                with h5py.File(os.path.join('../../', 'mitdb', 'train_ecg.hdf5'), 'r') as hdf:
                    self.x = hdf['x_train'][:]
                    self.y = hdf['y_train'][:]
            else:
                with h5py.File(os.path.join('../../', 'mitdb', 'test_ecg.hdf5'), 'r') as hdf:
                    self.x = hdf['x_test'][:]
                    self.y = hdf['y_test'][:]
        
        def __len__(self):
            return len(self.x)
        
        def __getitem__(self, idx):
            return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])
        
    batch_size = 32
    train_dataset = ECG(train=True)
    test_dataset = ECG(train=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    total_batch = len(train_loader)
    print(total_batch)
elif mode == 'CIFAR10':
    root_path = '../../data/'  # Replace with actual path to CIFAR10 data

    batch_size = 32
    cifar10_mean = (0.485, 0.456, 0.406)
    cifar10_std = (0.229, 0.224, 0.225)

    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=cifar10_mean, std=cifar10_std)
    ])
    train_dataset = CIFAR10(root=root_path, train=True, download=True, transform=transform)
    test_dataset = CIFAR10(root=root_path, train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    print("Loaded CIFAR10 datasets with batch size", batch_size)

Files already downloaded and verified
Files already downloaded and verified
Loaded CIFAR10 datasets with batch size 32


In [3]:
from torchvision.models import resnet18

if mode == 'ECG':
    class EcgClient(nn.Module):
        def __init__(self):
            super(EcgClient, self).__init__()
            self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
            self.relu1 = nn.ReLU()
            self.pool1 = nn.MaxPool1d(2)  # 64 x 16
            self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
            self.relu2 = nn.ReLU()
            self.pool2 = nn.MaxPool1d(2)  # 32 x 16
        
        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.pool1(x)
            x = self.conv2(x)
            x = self.relu2(x)
            x = self.pool2(x)
            x = x.view(-1, 32 * 16)
            return x
        
elif mode == 'CIFAR10':
    class AlexNetClient(nn.Module):
        def __init__(self):
            super(AlexNetClient, self).__init__()
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
                nn.Conv2d(64, 192, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
                nn.Conv2d(192, 384, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(384, 256, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
            )

        def forward(self, x):
            x = self.features(x)
            return x
    # class Resnet18_Any_Cut_Client(nn.Module): # M
    #     def __init__(self, config):
    #         super().__init__()
    #         self.cut_layer = config["cut_layer"]

    #         self.model = resnet18(weights = None)

    #         self.model = nn.ModuleList(self.model.children())
    #         self.model = nn.Sequential(*self.model)

    #     def forward(self, x):
    #         for i, layer in enumerate(self.model):
    #             if i > self.cut_layer:
    #                 break
    #             else:
    #                 x = layer(x)

    #         return x  

In [4]:
device = torch.device('cuda:0')
torch.cuda.get_device_name(0)
assert('cuda' in device.type)
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

cut_layer = 5
logits = 10

# config = {"cut_layer":cut_layer, "logits":logits}
ecg_client = AlexNetClient()
ecg_client.to(device)

# checkpoint = torch.load(f"../ecg_2conv/init_weight{mode}.pth")
# ecg_client.conv1.weight.data = checkpoint["conv1.weight"]
# ecg_client.conv1.bias.data = checkpoint["conv1.bias"]
# ecg_client.conv2.weight.data = checkpoint["conv2.weight"]
# ecg_client.conv2.bias.data = checkpoint["conv2.bias"]
epoch = 400
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = Adam(ecg_client.parameters(), lr=lr)
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data
host = 'localhost'
port = 10080
max_recv = 4096
s = socket.socket()
s.connect((host, port))

Training Process

In [5]:
from tqdm import tqdm
for e in range(epoch):
    print("Epoch {} - ".format(e+1), end='')
    
    for _, batch in tqdm(enumerate(train_loader)):
        x, label = batch
        x, label = x.to(device), label.to(device)
        optimizer.zero_grad()
        ecg_client = ecg_client.to(device)
        output = ecg_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        msg = pickle.dumps(msg)
        send_msg(s, msg)
        msg = recv_msg(s)
        client_grad = pickle.loads(msg)
        output.backward(client_grad)
        optimizer.step()

    # calculate test accuracy
    with torch.no_grad():
        for _, batch in enumerate(test_loader):
            x, label = batch
            x, label = x.to(device), label.to(device)
            client_output = ecg_client(x)
            msg = {
                'client_output': client_output,
                'label': label
            }
            msg = pickle.dumps(msg)
            send_msg(s, msg)
    
    msg = recv_msg(s)
    train_test_status = pickle.loads(msg)
    print(train_test_status)

Epoch 1 - 

1563it [00:16, 96.06it/s] 


loss: 1.6747, acc: 35.32% / test_loss: 0.2795, test_acc: 48.48%
Epoch 2 - 

1563it [00:15, 98.23it/s] 


loss: 1.2900, acc: 53.77% / test_loss: 0.2339, test_acc: 58.39%
Epoch 3 - 

...

1563it [00:13, 112.78it/s]


loss: 2.3028, acc: 9.65% / test_loss: 0.4611, test_acc: 10.00%
Epoch 303 - 

1563it [00:14, 104.80it/s]


loss: 2.3028, acc: 9.65% / test_loss: 0.4611, test_acc: 10.00%
Epoch 304 - 

644it [00:22, 98.89it/s] 

In [ ]:
print('Finished Training!')
print('Result is on the server side.')

Finished Training!
Result is on the server side.
